# Society of Mind

In [1]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import yaml


In [2]:
from community.data.datasets import get_datasets
from community.common.init import init_community, init_optimizers
from community.common.training import train_community

In [3]:
import warnings
#warnings.filterwarnings('ignore')

In [4]:
%load_ext autoreload
%autoreload 2

In [13]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 256
multi_loaders, double_loaders, single_loaders = get_datasets('../data', batch_size, use_cuda)

## Standard Community

### Community Initialization

In [14]:
agents_params_dict = {'n_agents' : 2,
                         'n_in' : 784,
                         'n_ins' : None,
                         'n_hid' : 100,
                         'n_layer' : 1,
                         'n_out' : 10,
                         'train_in_out': (True, False),
                         'use_readout': True,
                         'cell_type': str(nn.RNN),
                         'use_bottleneck': False,
                         'dropout': 0}

p_con = 1e-3

community = init_community(agents_params_dict, p_con, device=device)
community.nb_connections

{'01': tensor(10), '10': tensor(10)}

In [15]:
params = lr, gamma = 1e-3, 0.95
params_dict = {'lr' : lr, 'gamma' : gamma}

deepR_params = l1, gdnoise, lr, gamma, cooling = 1e-5, 1e-3, 1e-3, 0.95, 0.95
deepR_params_dict = {'l1' : l1, 'gdnoise' : gdnoise, 'lr' : lr, 'gamma' : gamma, 'cooling' : cooling}


optimizers, schedulers = init_optimizers(community, params_dict, deepR_params_dict)

In [ ]:
n

### Training

In [16]:
training_dict = {
    'n_epochs' : 2, 
    'task' : 'parity_digits',
    'global_rewire' : True, 
    'check_gradients' : False, 
    'reg_factor' : 0.,
    'train_connections' : True,
    'global_rewire' : True,
    'decision_params' : ('last', 'max'),
    'early_stop' : True ,
    'deepR_params_dict' : deepR_params_dict,
}

#pyaml.save(training_dict, '../community/common/default_train_dict.yml')

train_out = train_community(community, *double_loaders, optimizers, 
                            schedulers=schedulers, config=training_dict, device=device)
                            
results = train_out

Train Epoch::   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_losses, train_accs, test_losses, test_accs, deciding_agents, best_state = list(results.values())

### Results

In [ ]:
plt.plot(train_accs)
m = 1
epochs_slices = np.arange(0, len(train_accs)+1, m*len(double_loaders[0]))
max_per_epoch = [np.max(train_accs[epochs_slices[e]:epochs_slices[e+1]]) for e in range(len(epochs_slices)-1)]
#max_per_epoch.append(np.max(train_accs[epochs_slices[-1]:]))
plt.plot(epochs_slices[1:], max_per_epoch)


In [ ]:
plt.plot(train_losses)